In [1]:
#Defines directories and credent
execfile('theStuff.py')

import psycopg2
import getpass
import psycopg2.extras
import pandas as pd
import numpy as np

In [2]:

conn_string = "host='localhost' dbname=" + str(database) + " user=" + str(pgu) + " password=" + str(passpg)
conn= psycopg2.connect(conn_string)

In [3]:
# Identifies proper nouns in sentences containing word dam.  This can be used to query dam names, stream names, states and alike
# that may help to identify main dam removal(s) referenced in a paper.

def findDam(words): 
    import re
    dam_index = []
    
    for indx, x in enumerate(words):
        word = x.lower()
        match = re.match(r"dam", word) or re.match(r"dam-.*", word) or re.match(r".*-dam", word)
        if match:
            dam_index.append(indx)
    return dam_index
        
def properNouns(cursor):
    d = []
    #Requires cursor from postgres query of initial NLP data from UW database
    for sentence in cursor:
        #Identify fields within cursor
        docid = sentence[0]
        sentid = sentence[1]
        words = sentence[3]
        poses = sentence[4]
        index_num = len(poses)   # Set index for total number of items within sentence
        dam_index = findDam(words)
        ####################################################################################################
        ###Loop through each sentence item################################################################## 
        #if Proper Noun return docid, sentid, begin_index, end_index, and entire string of Proper Noun Words
        ###################################################################################################
        
        for i in xrange(index_num): 
            
            #try used to catch first iteration where end_index is not currently defined
            try:   
                # If catches all Proper Nouns that are more than one word, elif catches Proper Nouns = 1 word
                # if and elif require end index to be less than index so partial string of Proper Nouns are not captured 
                # where end_index is greater than index continue to next index value in sentence 
                if i > end_index and poses[i] == "NNP" and (poses[i+1] == 'NNP' or poses[i+1] == 'NN'):
                    end_index = i + 1
                    proper_noun = words[i]
                    while end_index < index_num and ((poses[end_index] == "NNP") or (poses[end_index] == "NN")):
                        proper_noun += (" " + words[end_index])
                        mention_id = "%s_%d_%d_%d" % (docid, sentid, i, end_index)
                        end_index += 1
                elif i > end_index and poses[i] == "NNP":
                    end_index = i 
                    mention_id = "%s_%d_%d_%d" % (docid, sentid, i, end_index)
                    proper_noun = words[i]
                else:
                    continue
            except:
                if poses[i] == "NNP" and (poses[i+1] == 'NNP' or poses[i+1] == 'NN'):
                    end_index = i + 1
                    proper_noun = words[i]
                    while end_index < index_num and ((poses[end_index] == "NNP") or (poses[end_index] == "NN")):
                        proper_noun += (" " + words[end_index])
                        mention_id = "%s_%d_%d_%d" % (docid, sentid, i, end_index)
                        end_index += 1
                elif poses[i] == "NNP":
                    end_index = i 
                    mention_id = "%s_%d_%d_%d" % (docid, sentid, i, end_index)
                    proper_noun = words[i]
                else: 
                    continue
            d.append({"mention_id": mention_id, 'docid': docid, 'sentid': sentid, 'begin_index': i, 'end_index': end_index, 'proper_noun': proper_noun, 'dam_index': dam_index})


            
        #Clear variables: mostly for rerunning in ipython notebook
        try:
            del end_index
        except:
            continue
        try:
            del i
        except:
            continue
    return d


In [4]:
#returns sentences with work dam in it
cursor = conn.cursor()
#cursor.execute("select distinct(b.*) from (select docid, sentid, unnest(words) as word from sentences) a left join sentences b on (a.docid=b.docid AND a.sentid=b.sentid) where (lower(a.word) = 'dam' OR lower(a.word) LIKE '%-dam' OR lower(a.word) LIKE 'dam-%' OR lower(a.word) = 'dams') ;")

#Small query return for testing purposes

cursor.execute("select * from public.sentences where docid = '57a8c3f9cf58f19c6886b244' and (sentid in (20,21,22,23,24,25,26,27));")
data = properNouns(cursor)
cursor.close()
      
df=pd.DataFrame(data)
df.head(150)


            

,begin_index,dam_index,docid,end_index,mention_id,proper_noun,sentid
0,22,[],57a8c3f9cf58f19c6886b244,22,57a8c3f9cf58f19c6886b244_21_22_22,Wipﬂi,21
1,32,[],57a8c3f9cf58f19c6886b244,32,57a8c3f9cf58f19c6886b244_21_32_32,Bellmore,21
2,42,[],57a8c3f9cf58f19c6886b244,42,57a8c3f9cf58f19c6886b244_21_42_42,Christie,21
3,56,[],57a8c3f9cf58f19c6886b244,56,57a8c3f9cf58f19c6886b244_21_56_56,Tonra,21
4,30,[],57a8c3f9cf58f19c6886b244,30,57a8c3f9cf58f19c6886b244_20_30_30,Naiman,20
5,5,[],57a8c3f9cf58f19c6886b244,8,57a8c3f9cf58f19c6886b244_26_5_7,Elsevier B.V. All,26
6,14,[37],57a8c3f9cf58f19c6886b244,14,57a8c3f9cf58f19c6886b244_22_14_14,Janetski,22
7,22,"[9, 23]",57a8c3f9cf58f19c6886b244,24,57a8c3f9cf58f19c6886b244_23_22_23,Elwha Dam,23
8,26,"[9, 23]",57a8c3f9cf58f19c6886b244,28,57a8c3f9cf58f19c6886b244_23_26_27,Elwha River,23
9,29,"[9, 23]",57a8c3f9cf58f19c6886b244,29,57a8c3f9cf58f19c6886b244_23_29_29,WA,23


In [5]:
f = directory + "/app-template/testing/removedDams20151214.csv"
removedDams = pd.read_csv(f)
merged = pd.merge(df, removedDams, how='inner', left_on='proper_noun', right_on='title')

In [6]:
merged.head()

,begin_index,dam_index,docid,end_index,mention_id,proper_noun,sentid,title,gnis_name,state
0,22,"[9, 23]",57a8c3f9cf58f19c6886b244,24,57a8c3f9cf58f19c6886b244_23_22_23,Elwha Dam,23,Elwha Dam,Elwha River,WA
1,9,[10],57a8c3f9cf58f19c6886b244,11,57a8c3f9cf58f19c6886b244_24_9_10,Elwha Dam,24,Elwha Dam,Elwha River,WA
2,35,[36],57a8c3f9cf58f19c6886b244,37,57a8c3f9cf58f19c6886b244_25_35_36,Elwha Dam,25,Elwha Dam,Elwha River,WA
